# Model Acceleration

Ngày nay bên cạnh nghiên cứu ra các mô hình học sâu chính xác hơn, nhanh hơn thì việc ứng dụng đưa các mô hình học sâu vào trong các sẩn phẩm cũng không kém phần quan trọng và gặp rất nhiều thách thức. Đặc biệt trong việc chuyển từ mô hình được viết bằng framework này sang framework khác vì mỗi thư viện có các hàm và kiểu dữ liệu khác nhau. Sau khi huấn luyện mô hình, chúng ta cần chuyển đổi mô hình sao cho tương thích với hardware sử dụng (e.g, Intel CPU, Nvidia GPU, ARM CPU, etc). Khi nghiên cứu thử nghiệm mô hình mình thường sử dụng pytorch vì dễ sử dụng và cộng đồng nghiên cứu cũng dùng torch nhiều rất tiện việc tra cứu. Tuy nhiên, khi triển khai thành sản phẩm thì trong một số công cụ lại chỉ hỗ trợ tensorflow do đó để sử dụng cần phải chuyển mô hình từ torch sang tensorflow. Lúc này chúng ta cần một dạng dữ liệu chuẩn cho các hàm cũng như các dạng dữ liệu (data types) để chuyển đổi. Và ONNX là chìa khóa có thể giải quyết tất cả vấn đề trên.

## Tăng tốc độ inference với Torch.compile
`torch.compile` làm cho Pytorch code chạy nhanh hơn bằng việc sử dụng JIT-compiling để convert code trở thành optimized kernels nhưng lại ít yêu cầu việc thay đổi code nên việc sử dụng tương đối đơn giản nhất.

### Thử nghiệm chạy 10 iteration with densnet121

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import transforms
import torch._dynamo
from PIL import Image

print(torch.cuda.is_available())

True


In [ ]:
def init_model():
    return torchvision.models.densenet121().to(torch.float32).cuda()

def generate_data(b):
    return (
        torch.randn(b, 3, 128, 128).to(torch.float32).cuda(),
        torch.randint(1000, (b,)).cuda(),
    )

def timed(fn):
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
    start.record()
    result = fn()
    end.record()
    torch.cuda.synchronize()
    return result, start.elapsed_time(end) / 1000

N_ITERS = 10
model = init_model()
torch._dynamo.reset()
model_opt = torch.compile(model, mode="reduce-overhead")

eager_times = []
for i in range(N_ITERS):
    inp = generate_data(16)[0]
    with torch.no_grad():
        _, eager_time = timed(lambda: model(inp))
    eager_times.append(eager_time)
    print(f"eager eval time {i}: {eager_time}")

print("~" * 10)
compile_times = []
for i in range(N_ITERS):
    inp = generate_data(16)[0]
    with torch.no_grad():
        _, compile_time = timed(lambda: model_opt(inp))
    compile_times.append(compile_time)
    print(f"compile eval time {i}: {compile_time}")
print("~" * 10)

eager_med = np.median(eager_times)
compile_med = np.median(compile_times)
speedup = eager_med / compile_med
assert(speedup > 1)
print(f"(eval) eager median: {eager_med}, compile median: {compile_med}, speedup: {speedup}x")
print("~" * 10)

eager eval time 0: 1.036719970703125
eager eval time 1: 0.02999740791320801
eager eval time 2: 0.02996428871154785
eager eval time 3: 0.029912832260131837
eager eval time 4: 0.02993699264526367
eager eval time 5: 0.029917695999145507
eager eval time 6: 0.02991971206665039
eager eval time 7: 0.020753696441650392
eager eval time 8: 0.01882111930847168
eager eval time 9: 0.018932416915893556
~~~~~~~~~~
compile eval time 0: 187.395515625
compile eval time 1: 0.7736571655273438
compile eval time 2: 0.016364959716796874
compile eval time 3: 0.016267263412475585
compile eval time 4: 0.01669526481628418
compile eval time 5: 0.018122495651245116
compile eval time 6: 0.016326751708984375
compile eval time 7: 0.0165479679107666
compile eval time 8: 0.016340991973876954
compile eval time 9: 0.016363519668579102
~~~~~~~~~~
(eval) eager median: 0.02991870403289795, compile median: 0.016456463813781737, speedup: 1.818051822764137x
~~~~~~~~~~


### Thử nghiệm với mô hình Pytorch ResNet50 cho classification.

In [ ]:
print(f"Non experimental in-tree backends: {torch._dynamo.list_backends()}")
print(f"Experimental or debug in-tree backends: {torch._dynamo.list_backends(None)}")
print(f"Mode in torch.compile: {torch._inductor.list_mode_options()}")

Non experimental in-tree backends: ['cudagraphs', 'inductor', 'onnxrt', 'openxla', 'openxla_eval', 'tvm']
Experimental or debug in-tree backends: ['aot_eager', 'aot_eager_decomp_partition', 'aot_eager_default_partitioner', 'aot_torchxla_trace_once', 'aot_torchxla_trivial', 'aot_ts', 'cudagraphs', 'dynamo_accuracy_minifier_backend', 'dynamo_minifier_backend', 'eager', 'eager_debug', 'inductor', 'non_leaf_compile_error_TESTING_ONLY', 'onnxrt', 'openxla', 'openxla_eval', 'pre_dispatch_eager', 'relu_accuracy_error_TESTING_ONLY', 'relu_compile_error_TESTING_ONLY', 'relu_runtime_error_TESTING_ONLY', 'torchxla_trace_once', 'torchxla_trivial', 'ts', 'tvm']
Mode in torch.compile: {'default': {}, 'reduce-overhead': {'triton.cudagraphs': True}, 'max-autotune-no-cudagraphs': {'max_autotune': True}, 'max-autotune': {'max_autotune': True, 'triton.cudagraphs': True}}


Determining the "best" backend depends on the specific use case and requirements. Here are some points to consider:

- **Performance**: inductor and cudagraphs are typically good choices for high performance on supported hardware.
- **Compatibility**: onnxrt (ONNX Runtime) and tvm can be good for deploying models across various platforms and devices.
- **Stability**: Stick to the non-experimental backends like inductor, onnxrt, openxla, and tvm for stable and production-ready applications.
- **Specific Needs**: Experimental backends may provide features that are not yet available in stable backends, useful for development and research purposes.

The mode attribute in `torch.compile(`) specifies different optimization strategies for compiling PyTorch models:
- **default**: Balances performance and overhead.
- **reduce-overhead**: Minimizes Python overhead using CUDA graphs, useful for small batches. May increase memory usage due to workspace memory caching. Works only for CUDA graphs that don’t mutate inputs. Use `TORCH_LOG=perf_hints` for debugging.
- **max-autotune**: Uses Triton-based matrix multiplications and convolutions with CUDA graphs enabled by default.
- **max-autotune-no-cudagraphs**: Similar to "max-autotune" but without CUDA graphs.

In [ ]:
model = torchvision.models.resnet50(weights="ResNet50_Weights.IMAGENET1K_V1", progress=True)
compiled_model = torch.compile(model, backend="eager")
model.eval()
compiled_model.eval()

In [ ]:
# Define the image preprocessing steps
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load an image
img_path = '/content/cock.png'
img = Image.open(img_path)
img_t = preprocess(img)
input_tensor = img_t.unsqueeze(0)

# Perform inference
with torch.inference_mode():
    output = model(input_tensor)
    output_ = compiled_model(input_tensor)
    print(torch.argmax(output, dim=1), torch.argmax(output_, dim=1))

tensor([7]) tensor([7])


In [ ]:
%%time
output = model(input_tensor)

CPU times: user 237 ms, sys: 51.9 ms, total: 289 ms
Wall time: 430 ms


In [ ]:
%%time
with torch.inference_mode():
  output = compiled_model(input_tensor)

CPU times: user 161 ms, sys: 2.9 ms, total: 163 ms
Wall time: 165 ms


## Tăng tốc độ với Torch Scripting

Khi mô hình gặp lỗi khi chuyển sang format ONNX, ta có thể sử dụng Torch Script. Đây là một phương pháp để chuyển mô hình Pytorch về dạng serializable và optmizable. Chỉ khi ở dạng serialized, script này có thể chạy không cần môi trường python, dependencies phức tạp. Ví dụ có thể inference [TorchScript models ở C++](https://pytorch.org/tutorials/advanced/cpp_export.html).

### Tracing and Scripting

### SwinTransformer-B Conversion
Ta sẽ thực hiện chuyển đổi SwinTransformer-B dưới dạng Torch Script.

In [ ]:
model = torchvision.models.swin_b(weights="Swin_B_Weights.IMAGENET1K_V1", progress=True)

Downloading: "https://download.pytorch.org/models/swin_b-68c6b09e.pth" to /root/.cache/torch/hub/checkpoints/swin_b-68c6b09e.pth
100%|██████████| 335M/335M [00:06<00:00, 52.6MB/s]


In [ ]:
torch.save(model, "swin-b.pt")

In [ ]:
model = torch.load('swin-b.pt')
scripted_model = torch.jit.script(model)
print(scripted_model)
scripted_model.save('scripted-swin-b.pt')

Benchmark accuracy: Xem [ImageNet1k class list](https://deeplearning.cms.waikato.ac.nz/user-guide/class-maps/IMAGENET/) để check độ chính xác

In [ ]:
def preprocess(path):
  augment = transforms.Compose([
      transforms.Resize(224),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ])

  image = Image.open(path)
  image = augment(image)
  return image.unsqueeze(0)

inputs = preprocess("/content/junco.png")

model.eval()
scripted_model.eval()

output = model(inputs)
output_ = scripted_model(inputs)
print(torch.argmax(output, dim=1), torch.argmax(output_, dim=1))

tensor([13]) tensor([13])


Benchmark Time

In [ ]:
inputs = [torch.randn(24, 3, 224, 224),
          torch.randn(24, 3, 224, 224),
          torch.randn(24, 3, 224, 224),
          torch.randn(24, 3, 224, 224)]

In [ ]:
%%timeit
with torch.inference_mode():
  for input in inputs:
    output = model(input)

In [ ]:
%%timeit
for input in inputs:
  output = scripted_model(inputs)

## Open Neural Network Exchange (ONNX)

**Giới thiệu về ONNX:** ONNX được xem là một ngôn ngữ đại diện cho công cụ toán học và thường được biểu diễn dưới dạng đồ thị (graph). Với ONNX, ta có thể xây dựng một quy trình deploy model độc lập hoàn toàn với các framework huấn luyện model (e.g, TensorFlow, Pytorch, etc).

**Cách hoạt động của ONNX**:
- Xây dụng một đồ thị ONNX nghĩa là sử dụng toán tử hay ngôn ngữ riêng của ONNX. Một đồ thị sẽ có những đỉnh (nodes), ta hình dung nodes trong đồ thị bằng phép toán đơn giản.

\begin{gather*}
y = x \times a  + c \\
y = r + c
\end{gather*}

- Ta có các nodes $x$, node $c$, node $r$ là kết quả trung gian của phép tính trên, và node $y$ là kết quả. Và các phép tính $\times$ và $+$ sẽ là các operator trên đường nối các nodes.
- Ta đều biết, để chạy mô hình, ta cần có môi trường cài đặt những dependencies cần thiết. Nhưng với ONNX, ta chỉ cần một runtime có sẵn bởi ONNX để chạy cái graph là một dãy những phép tính toán học trên những con số. Và runtime này có thể được code ở bất kỳ ngôn ngữ nào (e.g, C, java, python, javascript, C#, etc) để thực hiện inference với mô hình.
- Một điều lưu ý khi sử dụng ONNX là hạn chế sử dụng các câu lệnh `if/else/loop` vì ONNX thực chất chỉ là một cấu trúc cây (graph) thực hiện các phép tính toán trên tensors (ma trận/vector) nên sử dụng các câu lệnh trên sẽ làm cấu trúc cây trở nên chồng chéo phức tạp.

### Basic Example

In [ ]:
!pip install onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 27.1 MB/s eta 0:00:00


Chúng ta thử tạo một sơ đồ (graph) thể hiện phép tính sau: $Y = XA + B$. Đầu tiên, ta cần import các hàm cần thiết

In [ ]:
from onnx import TensorProto
from onnx.helper import (
    make_tensor_value_info,         # khởi tạo biến với shape và kiểu dữ liệu
    make_node,                      # Tạo một node đại diện cho phép tính
    make_graph,                     # Tạo sơ đồ tính toán dựa vào biến và phép tính trên
    make_model                      # Hàm cuối cùng để nhúng vào thêm metadata
)
from onnx.checker import check_model

Bước 1, ta cần khởi tạo các biến Tensor cần thiết với shape undefined (None):

In [ ]:
X = make_tensor_value_info('X', TensorProto.FLOAT, [None, None])
A = make_tensor_value_info('A', TensorProto.FLOAT, [None, None])
B = make_tensor_value_info('B', TensorProto.FLOAT, [None, None])
Y = make_tensor_value_info('Y', TensorProto.FLOAT, [None])

Bước 2, ta tạo các node đại diện cho phép tính sau khi đã có được các biến:

In [ ]:
node1 = make_node('MatMul', ['X', 'A'], ['XA'])
node2 = make_node('Add', ['XA', 'B'], ['Y'])

Ở `node1`, `X` và `A` là input, `XA` là output. Tương tự ở `node2`, `XA` và `B` là input còn `Y` là output.

Bước 3, ta tạo sơ đô tính toán (graph) với các biến và phép tính đã được khởi tạo sẵn ở trên để tổng quát hóa quá trình định nghĩa `input` và `output` cuối cùng là gì:

In [ ]:
graph = make_graph([node1, node2],  # nodes
                    'lr',           # a name
                    [X, A, B],      # inputs
                    [Y])            # outputs

Khởi tạo model từ graph trên:

In [ ]:
onnx_model = make_model(graph)
check_model(onnx_model)

Ta có thể in ra các attribute có trong biến model onnx_model này bằng cách:

In [ ]:
print(onnx_model.graph.input)
print(onnx_model.graph.output)
print(onnx_model.graph.node)

Một cách in khác cho đễ nhìn

In [ ]:
def shape2tuple(shape):
    return tuple(getattr(d, 'dim_value', 0) for d in shape.dim)

print('** inputs **')
for obj in onnx_model.graph.input:
    print("name=%r dtype=%r shape=%r" % (
        obj.name, obj.type.tensor_type.elem_type,
        shape2tuple(obj.type.tensor_type.shape)))

** inputs **
name='X' dtype=1 shape=(0, 0)
name='A' dtype=1 shape=(0, 0)
name='B' dtype=1 shape=(0, 0)


In [ ]:
print(onnx_model.graph.node)

[input: "X"
input: "A"
output: "XA"
op_type: "MatMul"
, input: "XA"
input: "B"
output: "Y"
op_type: "Add"
]


### Data Serialization

In [ ]:
import numpy
from onnx.numpy_helper import from_array, to_array
from onnx import TensorProto

numpy_tensor = numpy.array([0, 1, 4, 5, 3], dtype=numpy.float32)
print(type(numpy_tensor))

onnx_tensor = from_array(numpy_tensor)
print(type(onnx_tensor))

serialized_tensor = onnx_tensor.SerializeToString()
print(type(serialized_tensor))

with open("saved_tensor.pb", "wb") as f:
    f.write(serialized_tensor)

<class 'numpy.ndarray'>
<class 'onnx.onnx_ml_pb2.TensorProto'>
<class 'bytes'>


In [ ]:
with open("saved_tensor.pb", "rb") as f:
    serialized_tensor = f.read()
print(type(serialized_tensor))

onnx_tensor = TensorProto()
onnx_tensor.ParseFromString(serialized_tensor)
print(type(onnx_tensor))

numpy_tensor = to_array(onnx_tensor)
print(numpy_tensor)

<class 'bytes'>
<class 'onnx.onnx_ml_pb2.TensorProto'>
[0. 1. 4. 5. 3.]


In [ ]:
import onnx
import pprint
pprint.pprint([p for p in dir(onnx)
               if p.endswith('Proto') and p[0] != '_'])

['AttributeProto',
 'FunctionProto',
 'GraphProto',
 'MapProto',
 'ModelProto',
 'NodeProto',
 'OperatorProto',
 'OperatorSetIdProto',
 'OperatorSetProto',
 'OptionalProto',
 'SequenceProto',
 'SparseTensorProto',
 'StringStringEntryProto',
 'TensorProto',
 'TensorShapeProto',
 'TrainingInfoProto',
 'TypeProto',
 'ValueInfoProto']


In [ ]:
from onnx import load_tensor_from_string

with open("saved_tensor.pb", "rb") as f:
    serialized = f.read()
proto = load_tensor_from_string(serialized)
print(type(proto))

<class 'onnx.onnx_ml_pb2.TensorProto'>


### Initializer, default value

In [ ]:
import numpy
from onnx import numpy_helper, TensorProto
from onnx.helper import (
    make_model, make_node, make_graph,
    make_tensor_value_info)
from onnx.checker import check_model

def shape2tuple(shape):
    return tuple(getattr(d, 'dim_value', 0) for d in shape.dim)

X = make_tensor_value_info('X', TensorProto.FLOAT, [None, None])
A = make_tensor_value_info('A', TensorProto.FLOAT, [None, None])
B = make_tensor_value_info('B', TensorProto.FLOAT, [None, None])
Y = make_tensor_value_info('Y', TensorProto.FLOAT, [None])

node1 = make_node('MatMul', ['X', 'A'], ['XA'])
node2 = make_node('Add', ['XA', 'B'], ['Y'])

graph = make_graph([node1, node2], 'lr', [X, A, B], [Y])
onnx_model = make_model(graph)
check_model(onnx_model)

# The serialization
with open("linear_regression.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [ ]:
# initializers
value = numpy.array([0.5, -0.6], dtype=numpy.float32)
A = numpy_helper.from_array(value, name='A')

value = numpy.array([0.4], dtype=numpy.float32)
C = numpy_helper.from_array(value, name='C')

# the part which does not change
X = make_tensor_value_info('X', TensorProto.FLOAT, [None, None])
Y = make_tensor_value_info('Y', TensorProto.FLOAT, [None])
node1 = make_node('MatMul', ['X', 'A'], ['AX'])
node2 = make_node('Add', ['AX', 'C'], ['Y'])
graph = make_graph([node1, node2], 'lr', [X], [Y], [A, C])
onnx_model = make_model(graph)
check_model(onnx_model)

print('** initializer **')
for init in onnx_model.graph.initializer:
    print(init)

** initializer **
dims: 2
data_type: 1
name: "A"
raw_data: "\000\000\000?\232\231\031\277"

dims: 1
data_type: 1
name: "C"
raw_data: "\315\314\314>"



$$
Y = XA + B \\
y = \text{Add}(\text{MatMul}(X, \text{Transpose}(A)) + B)
$$

In [ ]:
# unchanged
X = make_tensor_value_info('X', TensorProto.FLOAT, [None, None])
A = make_tensor_value_info('A', TensorProto.FLOAT, [None, None])
B = make_tensor_value_info('B', TensorProto.FLOAT, [None, None])
Y = make_tensor_value_info('Y', TensorProto.FLOAT, [None])

# added
node_transpose = make_node('Transpose', ['A'], ['tA'], perm=[1, 0])

# unchanged except A is replaced by tA
node1 = make_node('MatMul', ['X', 'tA'], ['XA'])
node2 = make_node('Add', ['XA', 'B'], ['Y'])

# node_transpose is added to the list
graph = make_graph([node_transpose, node1, node2],
                   'lr', [X, A, B], [Y])
onnx_model = make_model(graph)
check_model(onnx_model)

# the work is done, let's display it...
print(onnx_model)

In [ ]:
import onnx
import pprint
pprint.pprint([k for k in dir(onnx.helper)
               if k.startswith('make')])

['make_attribute',
 'make_attribute_ref',
 'make_empty_tensor_value_info',
 'make_function',
 'make_graph',
 'make_map',
 'make_map_type_proto',
 'make_model',
 'make_model_gen_version',
 'make_node',
 'make_operatorsetid',
 'make_opsetid',
 'make_optional',
 'make_optional_type_proto',
 'make_sequence',
 'make_sequence_type_proto',
 'make_sparse_tensor',
 'make_sparse_tensor_type_proto',
 'make_sparse_tensor_value_info',
 'make_tensor',
 'make_tensor_sequence_value_info',
 'make_tensor_type_proto',
 'make_tensor_value_info',
 'make_training_info',
 'make_value_info']


### Metadata, opset

In [ ]:
from onnx import load, helper

with open("linear_regression.onnx", "rb") as f:
    onnx_model = load(f)

for field in ['doc_string', 'domain', 'functions',
              'ir_version', 'metadata_props', 'model_version',
              'opset_import', 'producer_name', 'producer_version',
              'training_info']:
    print(field, getattr(onnx_model, field))

doc_string 
domain 
functions []
ir_version 10
metadata_props []
model_version 0
opset_import [version: 21
]
producer_name 
producer_version 
training_info []


In [ ]:
with open("linear_regression.onnx", "rb") as f:
    onnx_model = load(f)

print("ir_version:", onnx_model.ir_version)
for opset in onnx_model.opset_import:
    print("opset domain=%r version=%r" % (opset.domain, opset.version))

ir_version: 10
opset domain='' version=21


In [ ]:
with open("linear_regression.onnx", "rb") as f:
    onnx_model = load(f)

del onnx_model.opset_import[:]
opset = onnx_model.opset_import.add() # comment this to see what happens
opset.domain = 'This is domain'
opset.version = 14

for opset in onnx_model.opset_import:
    print("opset domain=%r version=%r" % (opset.domain, opset.version))

opset domain='This is domain' version=14


In [ ]:
with open("linear_regression.onnx", "rb") as f:
    onnx_model = load(f)

onnx_model.model_version = 15
onnx_model.producer_name = "something"
onnx_model.producer_version = "some other thing"
onnx_model.doc_string = "documentation about this model"
prop = onnx_model.metadata_props

data = dict(key1="value1", key2="value2")
helper.set_model_props(onnx_model, data)

print(onnx_model)

### Subgraph: test and loops

In [ ]:
!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.1 MB/s eta 0:00:00


In [ ]:
import numpy
import onnx
from onnx.helper import (
    make_node, make_graph, make_model, make_tensor_value_info)
from onnx.numpy_helper import from_array
from onnx.checker import check_model
from onnxruntime import InferenceSession

# initializers
value = numpy.array([0], dtype=numpy.float32)
zero = from_array(value, name='zero')

# Same as before, X is the input, Y is the output.
X = make_tensor_value_info('X', onnx.TensorProto.FLOAT, [None, None])
Y = make_tensor_value_info('Y', onnx.TensorProto.FLOAT, [None])

# The node building the condition. The first one
# sum over all axes.
rsum = make_node('ReduceSum', ['X'], ['rsum'])
# The second compares the result to 0.
cond = make_node('Greater', ['rsum', 'zero'], ['cond'])

# Builds the graph is the condition is True.
# Input for then
then_out = make_tensor_value_info(
    'then_out', onnx.TensorProto.FLOAT, None)
# The constant to return.
then_cst = from_array(numpy.array([1]).astype(numpy.float32))

# The only node.
then_const_node = make_node(
    'Constant', inputs=[],
    outputs=['then_out'],
    value=then_cst, name='cst1')

# And the graph wrapping these elements.
then_body = make_graph(
    [then_const_node], 'then_body', [], [then_out])

# Same process for the else branch.
else_out = make_tensor_value_info(
    'else_out', onnx.TensorProto.FLOAT, [5])
else_cst = from_array(numpy.array([-1]).astype(numpy.float32))

else_const_node = make_node(
    'Constant', inputs=[],
    outputs=['else_out'],
    value=else_cst, name='cst2')

else_body = make_graph(
    [else_const_node], 'else_body',
    [], [else_out])

# Finally the node If taking both graphs as attributes.
if_node = onnx.helper.make_node(
    'If', ['cond'], ['Y'],
    then_branch=then_body,
    else_branch=else_body)

# The final graph.
graph = make_graph([rsum, cond, if_node], 'if', [X], [Y], [zero])
onnx_model = make_model(graph)
check_model(onnx_model)

# Let's freeze the opset.
del onnx_model.opset_import[:]
opset = onnx_model.opset_import.add()
opset.domain = ''
opset.version = 15
onnx_model.ir_version = 8

# Save.
with open("onnx_if_sign.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

# Let's see the output.
sess = InferenceSession(onnx_model.SerializeToString(),
                        providers=["CPUExecutionProvider"])

x = numpy.ones((3, 2), dtype=numpy.float32)
res = sess.run(None, {'X': x})

# It works.
print("result", res)
print()

# Some display.
print(onnx_model)

result [array([1.], dtype=float32)]

ir_version: 8
graph {
  node {
    input: "X"
    output: "rsum"
    op_type: "ReduceSum"
  }
  node {
    input: "rsum"
    input: "zero"
    output: "cond"
    op_type: "Greater"
  }
  node {
    input: "cond"
    output: "Y"
    op_type: "If"
    attribute {
      name: "else_branch"
      g {
        node {
          output: "else_out"
          name: "cst2"
          op_type: "Constant"
          attribute {
            name: "value"
            t {
              dims: 1
              data_type: 1
              raw_data: "\000\000\200\277"
            }
            type: TENSOR
          }
        }
        name: "else_body"
        output {
          name: "else_out"
          type {
            tensor_type {
              elem_type: 1
              shape {
                dim {
                  dim_value: 5
                }
              }
            }
          }
        }
      }
      type: GRAPH
    }
    attribute {
      name: "then_

### Function

In [ ]:
import numpy
from onnx import numpy_helper, TensorProto
from onnx.helper import (
    make_model, make_node, set_model_props, make_tensor,
    make_graph, make_tensor_value_info, make_opsetid,
    make_function)
from onnx.checker import check_model

new_domain = 'custom'
opset_imports = [make_opsetid("", 14), make_opsetid(new_domain, 1)]

# Let's define a function for a linear regression

node1 = make_node('MatMul', ['X', 'A'], ['XA'])
node2 = make_node('Add', ['XA', 'B'], ['Y'])

linear_regression = make_function(
    new_domain,            # domain name
    'LinearRegression',     # function name
    ['X', 'A', 'B'],        # input names
    ['Y'],                  # output names
    [node1, node2],         # nodes
    opset_imports,          # opsets
    [])                     # attribute names

# Let's use it in a graph.

X = make_tensor_value_info('X', TensorProto.FLOAT, [None, None])
A = make_tensor_value_info('A', TensorProto.FLOAT, [None, None])
B = make_tensor_value_info('B', TensorProto.FLOAT, [None, None])
Y = make_tensor_value_info('Y', TensorProto.FLOAT, [None])

graph = make_graph(
    [make_node('LinearRegression', ['X', 'A', 'B'], ['Y1'], domain=new_domain),
     make_node('Abs', ['Y1'], ['Y'])],
    'example',
    [X, A, B], [Y])

onnx_model = make_model(
    graph, opset_imports=opset_imports,
    functions=[linear_regression])  # functions to add)
check_model(onnx_model)

# the work is done, let's display it...
print(onnx_model)

### Function with attributes

In [ ]:
import numpy
from onnx import numpy_helper, TensorProto, AttributeProto
from onnx.helper import (
    make_model, make_node, set_model_props, make_tensor,
    make_graph, make_tensor_value_info, make_opsetid,
    make_function)
from onnx.checker import check_model

new_domain = 'custom'
opset_imports = [make_opsetid("", 14), make_opsetid(new_domain, 1)]

# Let's define a function for a linear regression
# The first step consists in creating a constant
# equal to the input parameter of the function.
cst = make_node('Constant',  [], ['B'])

att = AttributeProto()
att.name = "value"

# This line indicates the value comes from the argument
# named 'bias' the function is given.
att.ref_attr_name = "bias"
att.type = AttributeProto.TENSOR
cst.attribute.append(att)

node1 = make_node('MatMul', ['X', 'A'], ['XA'])
node2 = make_node('Add', ['XA', 'B'], ['Y'])

linear_regression = make_function(
    new_domain,            # domain name
    'LinearRegression',     # function name
    ['X', 'A'],             # input names
    ['Y'],                  # output names
    [cst, node1, node2],    # nodes
    opset_imports,          # opsets
    ["bias"])               # attribute names

# Let's use it in a graph.

X = make_tensor_value_info('X', TensorProto.FLOAT, [None, None])
A = make_tensor_value_info('A', TensorProto.FLOAT, [None, None])
B = make_tensor_value_info('B', TensorProto.FLOAT, [None, None])
Y = make_tensor_value_info('Y', TensorProto.FLOAT, [None])

graph = make_graph(
    [make_node('LinearRegression', ['X', 'A'], ['Y1'], domain=new_domain,
               # bias is now an argument of the function and is defined as a tensor
               bias=make_tensor('former_B', TensorProto.FLOAT, [1], [0.67])),
     make_node('Abs', ['Y1'], ['Y'])],
    'example',
    [X, A], [Y])

onnx_model = make_model(
    graph, opset_imports=opset_imports,
    functions=[linear_regression])  # functions to add)
check_model(onnx_model)

# the work is done, let's display it...
print(onnx_model)

### Evaluation and Runtime

In [ ]:
import numpy
from onnx import numpy_helper, TensorProto
from onnx.helper import (
    make_model, make_node, set_model_props, make_tensor,
    make_graph, make_tensor_value_info)
from onnx.checker import check_model
from onnx.reference import ReferenceEvaluator

X = make_tensor_value_info('X', TensorProto.FLOAT, [None, None])
A = make_tensor_value_info('A', TensorProto.FLOAT, [None, None])
B = make_tensor_value_info('B', TensorProto.FLOAT, [None, None])
Y = make_tensor_value_info('Y', TensorProto.FLOAT, [None])
node1 = make_node('MatMul', ['X', 'A'], ['XA'])
node2 = make_node('Add', ['XA', 'B'], ['Y'])
graph = make_graph([node1, node2], 'lr', [X, A, B], [Y])
onnx_model = make_model(graph)
check_model(onnx_model)

sess = ReferenceEvaluator(onnx_model)

x = numpy.random.randn(4, 2).astype(numpy.float32)
a = numpy.random.randn(2, 1).astype(numpy.float32)
b = numpy.random.randn(1, 1).astype(numpy.float32)
feeds = {'X': x, 'A': a, 'B': b}

print(sess.run(None, feeds))

[array([[-1.6823176 ],
       [-2.07294   ],
       [-0.3106615 ],
       [-0.17735608]], dtype=float32)]


In [ ]:
import numpy
from onnx import numpy_helper, TensorProto
from onnx.helper import (
    make_model, make_node, set_model_props, make_tensor,
    make_graph, make_tensor_value_info)
from onnx.checker import check_model
from onnx.reference import ReferenceEvaluator

X = make_tensor_value_info('X', TensorProto.FLOAT, [None, None])
A = make_tensor_value_info('A', TensorProto.FLOAT, [None, None])
B = make_tensor_value_info('B', TensorProto.FLOAT, [None, None])
Y = make_tensor_value_info('Y', TensorProto.FLOAT, [None])
node1 = make_node('MatMul', ['X', 'A'], ['XA'])
node2 = make_node('Add', ['XA', 'B'], ['Y'])
graph = make_graph([node1, node2], 'lr', [X, A, B], [Y])
onnx_model = make_model(graph)
check_model(onnx_model)

for verbose in [1, 2, 3, 4]:
    print()
    print(f"------ verbose={verbose}")
    print()
    sess = ReferenceEvaluator(onnx_model, verbose=verbose)

    x = numpy.random.randn(4, 2).astype(numpy.float32)
    a = numpy.random.randn(2, 1).astype(numpy.float32)
    b = numpy.random.randn(1, 1).astype(numpy.float32)
    feeds = {'X': x, 'A': a, 'B': b}

    print(sess.run(None, feeds))


------ verbose=1

[array([[ 1.6946731 ],
       [-0.08133245],
       [ 3.2676148 ],
       [ 1.3359237 ]], dtype=float32)]

------ verbose=2

MatMul(X, A) -> XA
Add(XA, B) -> Y
[array([[-0.25301543],
       [-0.03467366],
       [-0.35421404],
       [-0.18787482]], dtype=float32)]

------ verbose=3

 +I X: float32:(4, 2) in [-2.7870829105377197, 1.5124919414520264]
 +I A: float32:(2, 1) in [0.2143881767988205, 1.8981961011886597]
 +I B: float32:(1, 1) in [1.3698805570602417, 1.3698805570602417]
MatMul(X, A) -> XA
 + XA: float32:(4, 1) in [-1.3159455060958862, 2.936558485031128]
Add(XA, B) -> Y
 + Y: float32:(4, 1) in [0.05393505096435547, 4.30643892288208]
[array([[1.4402075 ],
       [4.306439  ],
       [3.020172  ],
       [0.05393505]], dtype=float32)]

------ verbose=4

 +I X: float32:(4, 2):0.9752460718154907,-1.4864572286605835,0.03714435547590256,0.6927376985549927,-0.7181068062782288...
 +I A: float32:(2, 1):[0.703309178352356, 0.443999320268631]
 +I B: float32:(1, 1):[0.43

## ONNX Runtime

**Khái niệm:** ONNX Runtime is a performance-focused engine for ONNX models, which inferences efficiently
across multiple platforms and hardware (Windows, Linux, and Mac and on
both CPUs and GPUs)

Ta không cần viết code với "ngôn ngữ" onnx từ đầu một cách thủ công lại như trên vì ta có thể sử dụng `ONNX Runtime` để chuyển đổi kiến trúc mô hình từ các framework `TensorFlow/Pytorch`. Hiện nay `ONNX Runtime` đã có thể tích hợp sẵn trong các thư viện ML/DL, ví dụ `keras2onnx`, `tf2onnx`, `torch.onnx`. Và việc inference đã được hỗ trợ bởi `onnxruntime.InferenceSession`.

### Sử dụng torch.onnx.export
Ta có hai bước cần thực hiện khi chuyển đổi sang onnx là 1.conversion và 2. inference. Ví dụ mẫu với mô hình DenseNet pretrained trên bộ dữ liệu ImageNet1K với thư viện torchvision.

#### Bước 1: Convert model

In [ ]:
model = torchvision.models.densenet161(weights="DenseNet161_Weights.IMAGENET1K_V1", progress=True)

Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/hub/checkpoints/densenet161-8d451a50.pth
100%|██████████| 110M/110M [00:02<00:00, 40.0MB/s]


**Constant folding** is an optimization technique used in compiler theory, including the context of neural network models. Here’s what it does:

- Identify Constants: The process scans the computational graph to identify operations that involve only constants.
- Compute at Export Time: Instead of keeping these constant computations as part of the graph, the values are precomputed during the export process.
- Simplify the Graph: The precomputed values replace the original operations in the graph, simplifying it and potentially reducing the computational load during runtime.

**Benefits**:
- Reduced Computation: By precomputing constant values, the model requires fewer computations during inference, which can lead to faster execution.
- Smaller Model Size: Simplifying the graph by removing unnecessary operations can reduce the overall size of the model.
- Optimization: It helps in optimizing the model for better performance on various hardware by eliminating redundant calculations.

In [ ]:
input = torch.randn(12, 3, 224, 224, requires_grad=True)

torch.onnx.export(
    model, input, "densenet161.onnx", export_params=False, opset_version=10,
    input_names=["input"], output_names=["output"], do_constant_folding=True,
    dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}}
)

#### Bước 2: Inference với converted model

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


Kiểm tra cuda version là 12 nên dùng ONNX Runtime Azure

In [ ]:
!pip install onnxruntime-gpu --extra-index-url https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/

In [ ]:
import onnxruntime as ort

# Preprocessing
def preprocess(path):
  augment = transforms.Compose([
      transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ])

  image = Image.open(path)
  image = augment(image)
  return image.unsqueeze(0).numpy()

inputs = preprocess("/content/junco.png")

# Prepare providers
providers = [
    'TensorrtExecutionProvider',
    'CUDAExecutionProvider',
    'CPUExecutionProvider'
]

ort_session = ort.InferenceSession("/content/densenet161.onnx", providers=providers)
inp = {ort_session.get_inputs()[0].name: inputs}
out = ort_session.run(None, inp)

# Postprocessing
def postprocess(out):
  idx = np.argmax(out[0])
  return idx

print(postprocess(out))

List được sắp xếp theo thứ tự ưu tiên, ngoài ra ta có thể specify thêm thông tin cho từng providers, đọc thêm tại [documentation](https://onnxruntime.ai/docs/execution-providers/CUDA-ExecutionProvider.html). Lưu ý để thực hiện inference với TensorRT, ta cần install đúng version của `cuda, cudnn, tensorrt`, tham khảo documentation này để tìm hiểu về Docker để cài TensorRT thành công. Dưới đây là một ví dụ về provider config cho TensorRT backend.

```python
providers = [
    ('TensorrtExecutionProvider', {
        'device_id': 0,                           # Select GPU to execute
        'trt_max_workspace_size': 2147483648,     # Set GPU memory usage limit
        'trt_fp16_enable': True,                  # Enable FP16 precision for faster inference  
        'trt_engine_cache_enable': True,
        'trt_engine_cache_path': 'Engine/onnx_models',
    }),
]
```

### Cách simpify mô hình onnx với onnxsim

Cơ chế khởi tạo sơ đồ phép tính (graph) của ONNX đôi khi có cấu trúc quá phức tạp và có những phần không cần thiết, ta có thể đơn giản hóa để làm gọn graph và làm mô hình nhẹ bằng thư viện [onnxsim](https://github.com/daquexian/onnx-simplifier).
- Cài thư viện

In [ ]:
!pip install onnxsim

Thực hiện trên terminal

In [ ]:
!onnxsim /content/densenet161.onnx /content/simplified_densenet161.onnx

Simplifying...
Finish! Here is the difference:
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃                    ┃ Original Model ┃ Simplified Model ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ AveragePool        │ 3              │ 3                │
│ BatchNormalization │ 82             │ 82               │
│ Concat             │ 82             │ 78               │
│ Constant           │ 569            │ 569              │
│ Conv               │ 160            │ 160              │
│ Flatten            │ 1              │ 1                │
│ Gemm               │ 1              │ 1                │
│ GlobalAveragePool  │ 1              │ 1                │
│ MaxPool            │ 1              │ 1                │
│ Relu               │ 161            │ 161              │
│ Model Size         │ 110.3MiB       │ 110.3MiB         │
└────────────────────┴────────────────┴──────────────────┘


In [ ]:
import onnx
from onnxsim import simplify

model = onnx.load("/content/densenet161.onnx")
model_simp, check = simplify(model)

assert check, "Simplified ONNX model could not be validated"
onnx.save(model_simp, "/content/densenet161_sim.onnx")

### ONNX with OpenVINO

In [ ]:
!pip install openvino --no-deps

In [ ]:
import openvino as ov

input = torch.tensor(12, 3, 224, 224)

core = ov.Core()
compiled_model = core.compile_model("model.onnx", "CPU")
infer_request = compiled_model.create_infer_request()

In [ ]:
# Warmup step
input_tensor = ov.Tensor(array=inputs, shared_memory=True)
infer_request.set_input_tensor(input_tensor)

In [ ]:
%%timeit
output_tensor = infer_request.infer()

## Precision and Quantization
Có hai cách để nén một mô hình nặng trở nên nhẹ hơn: low-rank precision và quantization.


### Precision:
- **Half-precision**: chuyển bộ trọng số $W$ từ kiểu dữ liệu `float32` (FP32) sang kiểu dữ liệu `float16` (FP16). Điều này giúp tối ưu một nửa bộ nhớ (reduced Memory Usage) và tăng hiệu năng (increased throughput) của hardware thích hợp với tính toán FP16, nhưng sẽ mất precision nên cần benchmark lại để đánh giá.
- **Mixed precision**: kết hợp một cách cân bằng giữa hai loại FP16 và FP32 để vừa tối ưu bộ nhớ nhưng vẫn tối thiểu sai số nhất có thể
    Sử dụng mixed precision với `neural-compressor.mix_precision` (intel)


Tham khảo bảng hỗ trợ của neural-compressor:
<table class="center">
<thead>
    <tr>
        <th>Framework</th>
        <th>Backend</th>
        <th>Backend Library</th>
        <th>Backend Value</th>
        <th>Support Device(cpu as default)</th>
        <th>Support BF16</th>
        <th>Support FP16</th>
    </tr>
</thead>
<tbody>
    <tr>
        <td rowspan="2" align="left">PyTorch</td>
        <td align="left">FX</td>
        <td align="left">FBGEMM</td>
        <td align="left">"default"</td>
        <td align="left">cpu</td>
        <td align="left">&#10004;</td>
        <td align="left">&#10006;</td>
    </tr>
    <tr>
        <td align="left">IPEX</td>
        <td align="left">OneDNN</td>
        <td align="left">"ipex"</td>
        <td align="left">cpu</td>
        <td align="left">&#10004;</td>
        <td align="left">&#10006;</td>
    </tr>
    <tr>
        <td rowspan="4" align="left">ONNX Runtime</td>
        <td align="left">CPUExecutionProvider</td>
        <td align="left">MLAS</td>
        <td align="left">"default"</td>
        <td align="left">cpu</td>
        <td align="left">&#10006;</td>
        <td align="left">&#10006;</td>
    </tr>
    <tr>
        <td align="left">TensorrtExecutionProvider</td>
        <td align="left">TensorRT</td>
        <td align="left">"onnxrt_trt_ep"</td>
        <td align="left">gpu</td>
        <td align="left">&#10006;</td>
        <td align="left">&#10006;</td>
    </tr>
    <tr>
        <td align="left">CUDAExecutionProvider</td>
        <td align="left">CUDA</td>
        <td align="left">"onnxrt_cuda_ep"</td>
        <td align="left">gpu</td>
        <td align="left">&#10004;</td>
        <td align="left">&#10004;</td>
    </tr>
    <tr>
        <td align="left">DnnlExecutionProvider</td>
        <td align="left">OneDNN</td>
        <td align="left">"onnxrt_dnnl_ep"</td>
        <td align="left">cpu</td>
        <td align="left">&#10004;</td>
        <td align="left">&#10006;</td>
    </tr>
    <tr>
        <td rowspan="2" align="left">Tensorflow</td>
        <td align="left">Tensorflow</td>
        <td align="left">OneDNN</td>
        <td align="left">"default"</td>
        <td align="left">cpu</td>
        <td align="left">&#10004;</td>
        <td align="left">&#10006;</td>
    </tr>
    <tr>
        <td align="left">ITEX</td>
        <td align="left">OneDNN</td>
        <td align="left">"itex"</td>
        <td align="left">cpu | gpu</td>
        <td align="left">&#10004;</td>
        <td align="left">&#10006;</td>
    </tr>  
    <tr>
        <td align="left">MXNet</td>
        <td align="left">OneDNN</td>
        <td align="left">OneDNN</td>
        <td align="left">"default"</td>
        <td align="left">cpu</td>
        <td align="left">&#10004;</td>
        <td align="left">&#10006;</td>
    </tr>
</tbody>
</table>

In [ ]:
!pip install neural-compressor

In [ ]:
from neural_compressor import mix_precision
from neural_compressor.config import MixedPrecisionConfig

Compress model trực tiếp từ model pytorch (chỉ có CPU)

In [ ]:
conf = MixedPrecisionConfig(
    backend="ipex",
    device="cpu",
    precisions="bf16",
)
torch_model = torch.load("/content/swin-b.pt")
converted_model = mix_precision.fit(torch_model, conf=conf)

Compress model.onnx với GPU

In [ ]:
conf = MixedPrecisionConfig(
    backend="onnxrt_cuda_ep",
    device="gpu",
    precisions="fp16",
) # chỉ sử dụng được với GPU cho model ỏ onnx format.

onnx_model = onnx.load("/content/densenet161_sim.onnx")
converted_model = mix_precision.fit(onnx_model, conf=conf)
converted_model.save("mixed_precision_densenet161_sim.onnx")

Compress model.onnx với CPU

In [ ]:
conf = MixedPrecisionConfig(
    backend="onnxrt_dnnl_ep",
    device="cpu",
    precisions="bf16",
)

onnx_model = onnx.load("/content/densenet161_sim.onnx")
converted_model = mix_precision.fit(onnx_model, conf=conf)
converted_model.save("mixed_precision_densenet_cpu.onnx")

### Quantization
Quantization là một kỹ thuật tối ưu nâng cao để tăng tốc độ inference/training. Nó giúp giảm số bit lưu trữ bằng cách chuyển số thực sang dạng số nguyên `int8`, `int4` nhưng không làm mất đi accuracy. Có thể phân loại quantization bằng hai cách.
- Chia theo đặc tính: Affine Quantization (asymmetric) và Scale Quantization (symmetric).
- Chia theo cách sử dụng: Post-Training Dynamic Quantization, Post-Training Static Quantization, Quantization-Aware Training.

#### Sử dụng
Sử dụng trực tiếp `torch.quantization`: \
**Post-Training Quantization**

In [ ]:
import torch

model = torch.load('/content/swin-b.pt')
model.eval()

quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

torch.save(quantized_model, 'quantized-swin-b.pt')

**Quantization-Aware Training**

```python
# Enable quantization-aware training
model.qconfig = torch.quantization.default_qconfig

# Convert the model to quantized version
quantized_model = torch.quantization.convert(model, inplace=False)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(quantized_model.parameters(), lr=0.01, momentum=0.9)

# Train the model
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # Get the inputs and labels
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = quantized_model(inputs)

        # Compute the loss
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()

        # Optimize
        optimizer.step()

# Evaluate the model
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = quantized_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

# Save the model
torch.save(quantized_model, 'mnist_model_quantized.pt')
```

Sử dụng `neural-compressor.quantization:`

In [ ]:
from neural_compressor.config import PostTrainingQuantConfig
from neural_compressor import quantization

Post-Training Quantization (w/o Accuracy Aware Tuning)
```python
val_dataset = ...
val_loader = ...
model = ...

conf = (PostTrainingQuantConfig())
quantized_model = quantization.fit(
    model=model,
    conf=conf,
    calib_dataloader=val_dataloader,
)
```
Post-Training Quantization (with Accuracy Aware Tuning)
```python
def validate(val_loader, model, criterion, args):
    ...
    return top1.avg

quantized_model = quantization.fit(
    model=model,
    conf=conf,
    calib_dataloader=val_dataloader,
    eval_func=validate,
)
```

Quantization-Aware Training
```python
from neural_compressor import QuantizationAwareTrainingConfig
from neural_compressor.training import prepare_compression

conf = QuantizationAwareTrainingConfig()
compression_manager = prepare_compression(model, conf)
compression_manager.callbacks.on_train_begin()
model = compression_manager.model
train_func(model)
compression_manager.callbacks.on_train_end()
compression_manager.save("./output")
```


Chọn backend cho vào bên trong `conf`:
    
<table class="center">
<thead>
    <tr>
        <th>Framework</th>
        <th>Backend</th>
        <th>Backend Library</th>
        <th>Backend Value</th>
        <th>Support Device</th>
    </tr>
</thead>
<tbody>
    <tr>
        <td rowspan="2" align="left">PyTorch</td>
        <td align="left">FX</td>
        <td align="left">FBGEMM</td>
        <td align="left">"default"</td>
        <td align="left">cpu</td>
    </tr>
    <tr>
        <td align="left">IPEX</td>
        <td align="left">OneDNN</td>
        <td align="left">"ipex"</td>
        <td align="left">cpu | xpu</td>
    </tr>
    <tr>
        <td rowspan="5" align="left">ONNX Runtime</td>
        <td align="left">CPUExecutionProvider</td>
        <td align="left">MLAS</td>
        <td align="left">"default"</td>
        <td align="left">cpu</td>
    </tr>
    <tr>
        <td align="left">TensorrtExecutionProvider</td>
        <td align="left">TensorRT</td>
        <td align="left">"onnxrt_trt_ep"</td>
        <td align="left">gpu</td>
    </tr>
    <tr>
        <td align="left">CUDAExecutionProvider</td>
        <td align="left">CUDA</td>
        <td align="left">"onnxrt_cuda_ep"</td>
        <td align="left">gpu</td>
    </tr>
    <tr>
        <td align="left">DnnlExecutionProvider</td>
        <td align="left">OneDNN</td>
        <td align="left">"onnxrt_dnnl_ep"</td>
        <td align="left">cpu</td>
    </tr>
    <tr>
        <td align="left">DmlExecutionProvider*</td>
        <td align="left">OneDNN</td>
        <td align="left">"onnxrt_dml_ep"</td>
        <td align="left">npu</td>
    </tr>
    <tr>
        <td rowspan="2" align="left">Tensorflow</td>
        <td align="left">Tensorflow</td>
        <td align="left">OneDNN</td>
        <td align="left">"default"</td>
        <td align="left">cpu</td>
    </tr>
    <tr>
        <td align="left">ITEX</td>
        <td align="left">OneDNN</td>
        <td align="left">"itex"</td>
        <td align="left">cpu | gpu</td>
    </tr>  
    <tr>
        <td align="left">MXNet</td>
        <td align="left">OneDNN</td>
        <td align="left">OneDNN</td>
        <td align="left">"default"</td>
        <td align="left">cpu</td>
    </tr>
</tbody>
</table>
<br>
<br>